In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS

In [2]:
# Product description: p tag
test_url = 'https://www.unileverfoodsolutions.co.th/en/product/knorr-slow-simmer-noodle-soup-powder-pork-flavoured-800-g-1-EN-197398.html'

test_html = requests.get(test_url)
test_soup = BS(test_html.content, 'html.parser')

test_soup.find('div', class_ = "pdp-section_top pdp-section_proplist clearfix").find('p').text.strip()

'Made for Chefs by Chefs, Knorr Slow Simmer Noodle Soup Powder is specially created for Thai noodle soup. Made with good quality pork spinal bone and flavours for a sweet bone taste, it is blended with herbs and spices to give a meaty soup in just 35 minutes'

In [3]:
# Product description: product top row description
test_url = 'https://www.unileverfoodsolutions.co.th/en/product/knorr-pronto-tomato-2-kg-1-EN-280689.html'

test_html = requests.get(test_url)
test_soup = BS(test_html.content, 'html.parser')

test_soup.find('div', class_ = "pdp-row_top").find('h3').text.strip()

'Made with sustainably grown Italian tomatoes.'

In [4]:
# Product description: product top row description
test_url = 'https://www.unileverfoodsolutions.co.th/en/product/knorr-pronto-tomato-2-kg-1-EN-280689.html'

test_html = requests.get(test_url)
test_soup = BS(test_html.content, 'html.parser')

for text in test_soup.find('ul', class_ = 'ufs-product-usplist').find_all('li'):
    print(text.text.strip())

No added MSG
Halal - Europe


In [8]:
# More product and allergy information
test_url = 'https://www.unileverfoodsolutions.co.th/en/product/knorr-cream-soup-base-1-kg-1-EN-114637.html'

test_html = requests.get(test_url)
test_soup = BS(test_html.content, 'html.parser')

for item in test_soup.find_all('div', class_ = 'pdp-row'):
    if None in (item.find('h3'), item.find('p')):
            continue
            
    print(item.find('h3').text)
    print(item.find('p').text)
    

Ingredients
Wheat flour 32%, Rice flour 21%, Skimmed milk powder 8%, Iodized Salt 7.3%, Non dairy creamer 6.1%, Sugar 5.8%, Steamed Chicken bone concentrated 0.8%, Bulking agent (Maltodextrin), Flavour enhancer (Monosodium glutamate), No Artificial flavours added, No Artificial colours added, No Preservatives added.

                       Allergen Information
Contain product from milk, and might contain soybean, celery, sulphides.
Nutrition Information


Description
Knorr Cream Soup Base is made with pasteurised skimmed milk powder, extract from chicken bone soup and onion to deliver Western Style taste in 10 minutes
Product Packaging
6 x 1 kg.
Container
Tub
Preparation
1. Dissolve 80g of Knorr Cream Soup Base in 1 litre of room temperature water then put it to boil 2. Add on other desired ingredients, whisk and simmer for another 10 minutes, and ready to serve.
Yield
12.5 L.
Shelf Life
6 months
Storage
Close pack tightly and store in a cool and dry place


## Main Code

## Try saving to pandas dataframe and export to csv

In [13]:
url_main = "https://www.unileverfoodsolutions.co.th"
url_products_catalog = url_main + "/en/products.html?page="

html_products_catalog = requests.get(url_products_catalog)
num_pages = 8

all_products_scraped = []

# Loop through each page that stores information of the products
for page_no in range(num_pages):
    url_page = url_products_catalog + str(page_no)

    # Request for html for product groups in current page and get soup
    html_page = requests.get(url_page)
    soup_page = BS(html_page.content, 'html.parser')
    
    # Get main product information
    for tile in soup_page.find_all('div', class_ = "product-tile js-product-tile col-span-4 product unavailable"):
        sku_number = tile.get('data-product-number')
        sku_name = tile.find('a').get('title')
        sku_url = url_main + str(tile.find('a').get('href')) 
        
        if None in (sku_number, sku_name, sku_url):
            continue
        
        # Go to product details page to scrape data there
        html_1Xproduct_catalog = requests.get(sku_url)
        soup_1Xproduct_catalog = BS(html_1Xproduct_catalog.content, 'html.parser')
        
        # Add full description of product if available
        product_description_full = []
        try:
            product_description_full.append(soup_1Xproduct_catalog.find('div', class_ = "pdp-section_top pdp-section_proplist clearfix").find('p').text.strip())
        except:
            product_description_full.append('')
        
        # Add short description of product from sub-header if available
        product_description_short = []
        try:
            product_description_short.append(soup_1Xproduct_catalog.find('div', class_ = "pdp-row_top").find('h3').text.strip())
        except:
            pass
        
        # Add short description of product if available
        try:
            for text in soup_1Xproduct_catalog.find('ul', class_ = 'ufs-product-usplist').find_all('li'):
                product_description_short.append(text.text.strip())
        except:
            product_description_short.append('')
        
        # Add product detailed information if available
        product_details = {}
        for item in soup_1Xproduct_catalog.find_all('div', class_ = 'pdp-row'):
            if None in (item.find('h3'), item.find('p')):
                    continue
            
            product_details[item.find('h3').text.strip()] = item.find('p').text.strip()
        
        # Save all data into dictionary and append to the main list 
        product_dict = {**{'sku_number': sku_number}, 
                        **{'sku_name': sku_name},
                        **{'sku_url': sku_url},
                        **{'full_description': product_description_full},
                        **{'short_description': product_description_short},
                        **product_details}
        
        all_products_scraped.append(product_dict)

all_products_scraped = pd.DataFrame(all_products_scraped)
            

In [25]:
all_products_scraped

,sku_number,sku_name,sku_url,full_description,short_description,Allergen Information,Nutrition Information,Product Packaging,Container,Benefits,Shelf Life,Ingredients,Description,Storage,Preparation,Yield,Tips
0,67056594,Knorr Pronto Tomato 2 kg,https://www.unileverfoodsolutions.co.th/en/pro...,[],[Made with sustainably grown Italian tomatoes....,"Tomato, Sunflower Oil, Salt, Sugar, Citric Aci...",,6 x 2 kg.,Tin can,Deliver authentic Itlian taste,24 months,NaN,NaN,NaN,NaN,NaN,NaN
1,21073122,BEST FOODS Real Mayonnaise 3.5 L,https://www.unileverfoodsolutions.co.th/en/pro...,[],"[The perfect base, nice color, retain shape., ...","Contain soybean, egg, mustard, and might conta...",,2 x 3.5 L.,Pouch,NaN,9 months,"Soybean oil 70%, Egg 11%, Distilled vinegar 3....",Best Foods Real Mayonnaise is a high quality m...,Keep in a cool & dry place. The Product is goo...,NaN,NaN,NaN
2,67035120,KNORR Intense Meaty Essence 1 kg,https://www.unileverfoodsolutions.co.th/en/pro...,[],[Knorr Intense Meaty Essence intensifies the f...,"Contains Egg, Milk, Fish and may contain Shrim...",,6 x 1 kg.,Bottle,NaN,15 months,"Salt (Non Iodized) 16%, Long-boiled chicken ex...",Knorr Intense Meaty Essence gives real chicken...,Close pack tightly and refrigerate after opening.,Shake well before use. Start with a few drops ...,83 servings,NaN
3,20062494,BEST FOODS Sweet Bakery Mayonnaise 910 g,https://www.unileverfoodsolutions.co.th/en/pro...,[],[Easy for decoration control and dispensing fr...,"Contain soybean, egg, mustard, and might conta...",,12 x 910 g.,Pouch,Consistency in sweet & mild taste for every ba...,6 months,"Sugar 26%, Soybean oil 13%, Distilled vinegar ...","Thai favourite sweet & mild taste, suitable fo...",The product is good for 2 months in refrigerat...,NaN,NaN,NaN
4,67595439,Knorr Beef Broth Base No Added MSG 1.5 kg,https://www.unileverfoodsolutions.co.th/en/pro...,[],"[Meaty taste & no added MSG, Made from real be...",No added MSG. No preservatives added. Halal ce...,,6 x 1.5 kg.,Tub,Meaty taste & no added MSG,9 Months,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,21069931,Sir Thomas J. Lipton Earl Grey Tea 2 g,https://www.unileverfoodsolutions.co.th/en/pro...,"[With lingering notes of Bergamot, this except...",[],Might contain lecithin from soy,,6 x 25 x 2 g.,Box,NaN,24 months,Black Tea 95% Nature-Identical Flavour added,"With lingering notes of Bergamot, this excepti...",Store in a cool and dry place,Place one tea bag for each person in a cup or ...,NaN,"Add milk, sugar or honey if desired"
81,21069560,Sir Thomas J. Lipton Green Tea with Jasmine Te...,https://www.unileverfoodsolutions.co.th/en/pro...,[Lift your spirits with this green tea delicat...,[],Might contain lecithin from soy,,6 x 25 x 2 g.,Box,NaN,24 months,"Green Tea 99%, Jasmine Flowers 1%",Lift your spirits with this green tea delicate...,Store in a cool and dry place,Place one tea bag for each person in a cup or ...,NaN,"Add milk, sugar or honey if desired"
82,21076142,KNORR SELECTIONS Tomato Ketchup 5.2 kg,https://www.unileverfoodsolutions.co.th/en/pro...,[Made for chefs by chefs. Knorr Tomato Ketchup...,[],NaN,,3 x 5.2 kg.,Gallon,NaN,6 months,"Tomato paste 23%, Sugar 23%, Distilled vinegar...",Made for chefs by chefs. Knorr Tomato Ketchup ...,The product is good for a month in refrigerato...,NaN,NaN,NaN
83,21069590,Sir Thomas J. Lipton English Breakfast Tea 2.4 g,https://www.unileverfoodsolutions.co.th/en/pro...,[Round up a heavy morning meal with this robus...,[],Might contain lecithin from soy,,6 x 25 x 2.4 g.,Box,NaN,24 months,Black Tea 100%,Round up a heavy morning meal with this robust...,Store in a cool and dry place,Place one tea bag for each person in a cup or ...,NaN,NaN


In [14]:
all_products_scraped.to_csv("check.csv")